In [ ]:
!pip install langchain-google-genai langchain -U -q   langchain-tavily

In [ ]:
import os
from getpass import getpass

if os.environ.get("GOOGLE_API_KEY") is None:
  os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API key: ")

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass("Tavily API key:\n")

Tavily API key:
··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_tavily import TavilySearch
from langchain_core.prompts import PromptTemplate
parser = StrOutputParser()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.4)

# response_check = llm.invoke("What is the capital of France?")
# print(response_check)



In [ ]:
#tavily setup



tool = TavilySearch(
    max_results=10,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

def tavilyResult(query:str) -> str:
  response = tool.invoke({"query": query})
  # print(response, "response")

  content = ""
  for result in response.get("results", []):
      content += result.get("content", "") + "\n\n"
  content.strip()
  return content

search_result = tavilyResult("What happened at the last IPL 2025")

In [ ]:
#social media templates

from langchain_core.runnables import RunnableParallel

linkedIn_template = PromptTemplate.from_template("""

From the {search_result}, create a LinkenIn style post.
""")

twitter_template = PromptTemplate.from_template("""

From the {search_result}, create a twitter style post.
""")

instagram_template = PromptTemplate.from_template("""

From the {search_result}, create a instagram style post.
""")


linkedIn_chain = linkedIn_template | llm | parser
twitter_chain = twitter_template | llm | parser
instagram_chain = instagram_template | llm | parser

# test_linked_chain = linkedIn_chain.invoke({"search_result": search_result})
# test_twitter_chain = twitter_chain.invoke({"search_result": search_result})
# test_instagram_chain = instagram_chain.invoke({"search_result": search_result})


# print("---------------LinkednInPost------------------")
# print(test_linked_chain)


# print("---------------TwitterPost------------------")
# print(test_twitter_chain)
# print("---------------InstagramPost------------------")
# print(test_instagram_chain )

parallel_chain = RunnableParallel({
    "linkedin": linkedIn_chain,
    "twitter": twitter_chain,
    "instagram": instagram_chain
})

draft = parallel_chain.invoke({"search_result": search_result})

# print(draft, "draft")

for key, value in draft.items():
    print(f"{key}: {value}")




linkedin: Here's a LinkedIn-style post based on the provided information:

---

**IPL 2025: A Season of Unprecedented Challenges, Resilience, and Heartbreak**

The 18th season of the Indian Premier League (IPL 2025) has concluded, leaving behind a narrative unlike any other in its history. This year, the league navigated through extraordinary circumstances, showcasing both the unifying power of sport and the profound impact of real-world events.

**A Pause Amidst Geopolitical Tensions:**
The season began with an immediate and unforeseen suspension. Escalating military tensions between India and Pakistan led to an initial one-week pause, quickly evolving into an indefinite suspension. Matches, including the Punjab Kings vs. Mumbai Indians fixture, were impacted, with one even shifted due to airport closures. BCCI officials, including Chairman Arun Dhumal, worked tirelessly behind the scenes, looking at possibilities for resumption following a ceasefire announcement.

**The Return to the

In [ ]:
from langchain.tools import Tool
from langchain_core.runnables import RunnableParallel

# Define the Tavily search tool
tavily_tool = Tool(
    name="TavilySearch",
    func=tavilyResult,
    description="useful for when you need to answer questions about current events"
)

# Combine social media chains using RunnableParallel
social_media_generator = RunnableParallel(
    linkedin=linkedIn_chain,
    twitter=twitter_chain,
    instagram=instagram_chain
)

# Although not strictly a "tool" in the traditional agent sense,
# we can think of this RunnableParallel as a tool that takes
# search results and outputs all social media posts simultaneously.
# We might use this after the agent has used the search tool.

print("Tools defined: tavily_tool, social_media_generator")

ImportError: cannot import name 'Tool' from 'langchain.tools' (/usr/local/lib/python3.12/dist-packages/langchain/tools/__init__.py)